In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
import pandas as pd
import pickle
from keras.models import load_model
from sklearn.metrics import accuracy_score
from keras.preprocessing.sequence import pad_sequences

def get_accuracy(data_path, tokenizer_path, model_path):
  data = pd.read_csv(data_path)
  ho_x_test = data['Tweet']
  ho_y_test = data['Sentiment']
  # load tokenizer: 
  # That's the one we will use to vectorize our data where we want to get the prediction
  with open(tokenizer_path, 'rb') as handle:
      tokenizer = pickle.load(handle)
  # load rnn model
  rnn_model = load_model(model_path)
  ho_test_sequences = tokenizer.texts_to_sequences(ho_x_test)
  padded_ho_test_sequences = pad_sequences(ho_test_sequences, maxlen=35)
  ho_y_pred = rnn_model.predict(padded_ho_test_sequences, verbose=1, batch_size=2048)
  ho_y_pred = pd.DataFrame(ho_y_pred, columns=['prediction'])
  ho_y_pred['prediction'] = ho_y_pred['prediction'].map(lambda p: 1 if p >= 0.5 else 0)
  return accuracy_score(ho_y_test, ho_y_pred)*100
  


print('Hydro Ottawa dataset accuracy: ', get_accuracy('/content/drive/My Drive/SentimentAnalysis/rnn/data/HydroOttawaAnnotatedData.csv',
             '/content/drive/My Drive/SentimentAnalysis/rnn/tokenizers/tokenizer.pickle',
             '/content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-glove-model-01-0.8362.hdf5'))

print('Manually created dataset accuracy: ', get_accuracy('/content/drive/My Drive/SentimentAnalysis/rnn/data/CompiledTweets.csv',
             '/content/drive/My Drive/SentimentAnalysis/rnn/tokenizers/tokenizer.pickle',
             '/content/drive/My Drive/SentimentAnalysis/rnn/models/rnn-glove-model-01-0.8362.hdf5'))


Using TensorFlow backend.


238/238 [==============================] - 1s 3ms/step
Hydro Ottawa dataset accuracy:  73.52941176470588
443/443 [==============================] - 1s 2ms/step
Manually created dataset accuracy:  89.61625282167043
